Sharon Kim
SBA 345: Apache Spark (PySpark)

Section One - CompanyABC Stock Data

In [2]:
#1.1

import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

spark_df = spark.read.load("C:\\Users\\SGKsk\\Downloads\\CompanyABC_stock.csv", format="csv", header = True,inferSchema = True)

In [3]:
spark_df.columns
spark_df.printSchema()
spark_df.describe().show()
spark_df.show()


root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.3899989999

In [5]:
#1.2 print first five rows

spark_df.head(5)

[Row(Date=datetime.date(2012, 1, 3), Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996),
 Row(Date=datetime.date(2012, 1, 4), Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475),
 Row(Date=datetime.date(2012, 1, 5), Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539),
 Row(Date=datetime.date(2012, 1, 6), Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922),
 Row(Date=datetime.date(2012, 1, 9), Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)]

In [8]:
#1.3: Create a new Dataframe column called “HV Ratio,” which will stimulate the ratio of the High price versus the total Volume of stock that was traded for a day. 

#new_column_df = spark_df.withColumn('HV Ratio', spark_df.High / spark_df.Volume)
#new_column_df.show()

new_column_df = spark_df.withColumn('HV Ratio', spark_df['High'] / spark_df['Volume']).select('HV Ratio').show()


+--------------------+
|            HV Ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows



In [10]:
#1.4: Find out on what day the stock price was the highest. (Hint: use the High column.)

spark_df.select("High", "Date").sort(spark_df['High'].desc()).show()

#The stock price was the highest on Jan 13, 2015.

+-----------------+----------+
|             High|      Date|
+-----------------+----------+
|        90.970001|2015-01-13|
|90.66999799999999|2015-01-08|
|        90.389999|2015-01-09|
|        90.309998|2015-01-12|
|        89.260002|2015-01-23|
|        89.160004|2015-01-26|
|            88.68|2015-01-07|
|        88.519997|2015-01-14|
|        88.459999|2015-01-27|
|        88.400002|2015-01-22|
|        88.230003|2015-01-28|
|        88.089996|2014-11-28|
|             88.0|2015-02-06|
|        87.779999|2015-01-15|
|        87.720001|2015-01-29|
|        87.699997|2015-01-20|
|        87.459999|2015-01-16|
|        87.440002|2014-12-31|
|        87.410004|2015-02-10|
|        87.360001|2015-01-30|
+-----------------+----------+
only showing top 20 rows



In [12]:
#1.5: What is the average (mean) closing price? (Hint: Use the Close column.)

from pyspark.sql.functions import mean,avg,max,min

spark_df.select(mean("Close")).show()

#The average closing price is ~$72.39.

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



In [13]:
#1.6: What is the maximum and minimum volume of stock traded? (Hint: Use the Volume column).

spark_df.select(max("Volume"),min("Volume")).show()

#The maximum and minimum volume of stock traded is 80898100 and 2094900, respectively.

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+



In [27]:
#1.7: For how many days was the closing value less than 70 dollars? (Hint: Use the count() method.)

#spark_70_df = spark_df.select('Date', 'Close').filter(spark_df['Close'] < 70).show(400)
spark_df.select('Date', 'Close').filter(spark_df['Close'] < 70).count()

397

In [37]:
#1.8: What percentage of the time were the High greater than 80 dollars? 

#spark_df.count() #1258
#spark_df.select('High').filter(spark_df['High'] > 80).count() #115
#print(str(115/1258 * 100) + ("%"))
print(str((spark_df.select('High').filter(spark_df['High'] > 80).count() / spark_df.count()) * 100) + ("%"))

#The High was greater than $80 ~9.14% of the time.

9.141494435612083%


In [41]:
#1.9: Create a database named CompanyABC_DB using SQL (Workbench). 

#done in SQL Workbench

In [45]:
#1.10: Load/Write CompanyABC stock.csv file data into CompanyABC_DB database from SparkSQL Dataframe. You can specify any table name for that file.

#df = spark.createDataFrame(data = "C:\\Users\\SGKsk\\Downloads\\CompanyABC_stock.csv")

df = spark.read.load("C:\\Users\\SGKsk\\Downloads\\CompanyABC_stock.csv", format="csv", header = True ,inferSchema = True)


df.write.format("jdbc") \
 .mode("append") \
 .option("url", "jdbc:mysql://localhost:3306/companyabc_db") \
 .option("dbtable", "companyabc_db.Table_forSpark") \
 .option("user", "root") \
 .option("password", "password") \
 .save()


Section Two - CompanyABC Sales Data

In [58]:
#2.1: Load/Read both CompanyABC sales datasets (Sales_April_2019.csv and Sales_February_2019.csv) into a single SparkSQL Dataframe ( hint read() ).

#df_multiple = spark.read.load("C:\\Users\\SGKsk\\Downloads\\Sales_February_2019.csv", "C:\\Users\\SGKsk\\Downloads\\Sales_April_2019.csv", format="csv", header = True ,inferSchema = True)


# Reading multiple CSV files into a DataFrame
df_multiple = spark.read.csv([
 "C:\\Users\\SGKsk\\Downloads\\Sales_February_2019.csv",
 "C:\\Users\\SGKsk\\Downloads\\Sales_April_2019.csv"
 ], header = True,)
df_multiple.show()

+--------+--------------------+----------------+----------+---------------+--------------------+
|Order ID|             Product|Quantity Ordered|Price Each|     Order Date|    Purchase Address|
+--------+--------------------+----------------+----------+---------------+--------------------+
|  176558|USB-C Charging Cable|               2|     11.95| 4/19/2019 8:46|917 1st St, Dalla...|
|  176559|Bose SoundSport H...|               1|     99.99| 4/7/2019 22:30|682 Chestnut St, ...|
|  176560|        Google Phone|               1|       600|4/12/2019 14:38|669 Spruce St, Lo...|
|  176560|    Wired Headphones|               1|     11.99|4/12/2019 14:38|669 Spruce St, Lo...|
|  176561|    Wired Headphones|               1|     11.99| 4/30/2019 9:27|333 8th St, Los A...|
|  176562|USB-C Charging Cable|               1|     11.95|4/29/2019 13:03|381 Wilson St, Sa...|
|  176563|Bose SoundSport H...|               1|     99.99|  4/2/2019 7:46|668 Center St, Se...|
|  176564|USB-C Charging Cable

In [59]:
df_multiple.columns
df_multiple.printSchema()
df_multiple.describe().show()
df_multiple.show()

root
 |-- Order ID: string (nullable = true)
 |-- Product: string (nullable = true)
 |-- Quantity Ordered: string (nullable = true)
 |-- Price Each: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Purchase Address: string (nullable = true)

+-------+------------------+------------+-------------------+------------------+--------------+--------------------+
|summary|          Order ID|     Product|   Quantity Ordered|        Price Each|    Order Date|    Purchase Address|
+-------+------------------+------------+-------------------+------------------+--------------+--------------------+
|  count|             30328|       30328|              30328|             30328|         30328|               30328|
|   mean|173816.63355904212|        NULL| 1.1239966969446737| 183.7621383980421|          NULL|                NULL|
| stddev|14902.179833784816|        NULL|0.43431295924749175|328.79347229907665|          NULL|                NULL|
|    min|            150502|20in 

In [61]:
#2.2. If you use the above command on the sales dataset, you will notice that each Order has “Price Each” and “Quantity Ordered” columns, but the “Total price” is missing.  See below:
#Now, create a new Spark Dataframe column called “Total price” and find the “Total price” of the Order for the combined sales file as shown in the screenshot

#new_column_df_multiple = df_multiple.withColumn('Total Price', df_multiple['Price Each'] * df_multiple['Quantity Ordered']).select('Total Price').show()
total_price_df = df_multiple.withColumn('Total Price', df_multiple['Price Each'] * df_multiple['Quantity Ordered'])
total_price_df.show()

+--------+--------------------+----------------+----------+---------------+--------------------+-----------+
|Order ID|             Product|Quantity Ordered|Price Each|     Order Date|    Purchase Address|Total Price|
+--------+--------------------+----------------+----------+---------------+--------------------+-----------+
|  176558|USB-C Charging Cable|               2|     11.95| 4/19/2019 8:46|917 1st St, Dalla...|       23.9|
|  176559|Bose SoundSport H...|               1|     99.99| 4/7/2019 22:30|682 Chestnut St, ...|      99.99|
|  176560|        Google Phone|               1|       600|4/12/2019 14:38|669 Spruce St, Lo...|      600.0|
|  176560|    Wired Headphones|               1|     11.99|4/12/2019 14:38|669 Spruce St, Lo...|      11.99|
|  176561|    Wired Headphones|               1|     11.99| 4/30/2019 9:27|333 8th St, Los A...|      11.99|
|  176562|USB-C Charging Cable|               1|     11.95|4/29/2019 13:03|381 Wilson St, Sa...|      11.95|
|  176563|Bose Soun

In [62]:
#2.3: Load/Write sales data from SparkSQL Dataframe into CompanyABC_DB database. You can specify any name to the table. Remember “Total price” must be recorded. Your result will look similar /closer to the screenshot below

total_price_df.write.format("jdbc") \
 .mode("append") \
 .option("url", "jdbc:mysql://localhost:3306/companyabc_db") \
 .option("dbtable", "companyabc_db.Sales_data") \
 .option("user", "root") \
 .option("password", "password") \
 .save()